In [1]:
import pandas as pd
from actionrules.actionRulesDiscovery import ActionRulesDiscovery

In [81]:
emp_data = {'EmployeeID' : [1, 2, 3, 4, 5, 6],
        'Position' : ['Developer', 'Developer', 'Developer', 'Developer', 'Developer','Developer'],
        'Salary' : ['Medium', 'Medium', 'Medium', 'High', 'High', 'High'],
        'Stayed' : ['Yes', 'No', 'No', 'Yes', 'Yes', 'No']}
data = pd.DataFrame(data=emp_data).set_index('EmployeeID')
data

,Position,Salary,Stayed
EmployeeID,,,
1,Developer,Medium,Yes
2,Developer,Medium,No
3,Developer,Medium,No
4,Developer,High,Yes
5,Developer,High,Yes
6,Developer,High,No


In [82]:
util_data = {0 : ['Salary_medium', 'Salary_high', 'Stayed_yes', 'Stayed_no'], 1 : [500, 1000, 700, 0]}
utility = pd.DataFrame(data=util_data).set_index(0)
utility

,1
0,
Salary_medium,500
Salary_high,1000
Stayed_yes,700
Stayed_no,0


In [79]:
actionRulesDiscovery = ActionRulesDiscovery()
actionRulesDiscovery.load_pandas(data)
actionRulesDiscovery.fit(stable_attributes=['Position'],
                         flexible_attributes=['Salary'],
                         consequent='Stayed',
                         conf=40,
                         supp=3,
                         desired_classes=['Yes'],
                         utility_source=utility,
                         min_profit=-300
                         )

In [80]:
actionRulesDiscovery.get_pretty_action_rules()

["If attribute 'Position' is 'developer', attribute 'Salary' value 'medium' is changed to 'high', then 'Stayed' value 'No' is changed to 'Yes' with support: 0.3333333333333333, confidence: 0.4444444444444444 and uplift: 0.16666666666666663. Profit of the action is -266.66666666666674."]

## General profit formula:
### profit = ((utility_positiveTarget - utility_negativeTarget) * (confidence_after + confidence_before - 1)) - (utility_flexAttributesAfter - utility_flexAttributesBefore)

Action rule has part before and after - each part has its own profit value. These two partial profit values are substracted and this profit value is measured against the *min_profit* threshold.

The antecedent part of the rule (the utility of the action - change in flexible attributes) represents the variable cost part while calculating the profit. In the example regarding the developers and their salaries, the flexible attribute is *Salary*. The respective utilities of the salary attribute values are 500 for *medium* and 1000 for *high*. The result action rule suggests raising the salary from *medium* to *high*, therefore the cost part of the rule is 1000 - 500 = 500. This number represents the cost of the action (e.g. how much money has to be paid to make concerned employees stay - *Stayed_no* -> *Stayed_yes*).

The consequent part of the rule (desired change in the target class) represents the benefit while calculating the profit. The attribute in the example which should be affected by action is *Stayed*. The utility value for the desired target attribute value *Yes* is 700 - for *No* is 0. These two numbers are substracted 700 - 0 = 700 and the substraction is multiplied by the sum of confidences of the rules before and after substracted by 1. 

The action (cost part) has to be performed fo every developer that already does not have high salary as there is no way in identifying which ones are the ones, that would be convinced by the salary raise to stay. However, only those who actually stay after the executed action are the ones that yields the utility.

The result action rule shows, that raising salaries for developers and them not leaving the company is in fact not profitable.